In [ ]:
# ans = []
# for (int i = 0; i < paint.size(); i++) {
#     start = paint[i][0];
#     end = paint[i][1] - 1;
#     ans.append(end - start + 1 - segment_tree_query(start, end)); # query the sum of this range, takes O(logN) time.
#     segment_tree_update(start, end, 1); # update all value in this range to be 1, which also takes O(logN) time.
# }
# return ans;  
    
    # segment tree node number:
    # Since the constructed tree is always a full binary tree with n leaves, 
    # there will be n-1 internal nodes. So the total number of nodes will be 2*n – 1.
    
    
class SegmentTree:
    def __init__(self, N: int):
        self.tree = [0] * N
    
    def increment(self, qlo: int, qhi: int, lo: int, hi: int, pos: int) -> int:
        total = hi - lo + 1
        
        if self.tree[pos] == total:     # if the node is full, return 0
            return 0
        
        if qlo <= lo <= hi <= qhi:      # case 1, total overlap
            missing = total - self.tree[pos]
            self.tree[pos] = total
            return missing
        elif qlo > hi or qhi < lo:      # case 2, NO overlap
            return 0
        
        mid = lo + (hi - lo) // 2       # case 3, partial overlap
        left = self.increment(qlo, qhi, lo, mid, 2 * pos + 1)
        right = self.increment(qlo, qhi, mid + 1, hi, 2 * pos + 2)
        self.tree[pos] += left + right
        return left + right
                

class Solution:
    def amountPainted(self, paint: List[List[int]]) -> List[int]:
        
        N = max(r for _, r in paint)
        
        tree = SegmentTree(4 * N)
        worklog = []
        
        for qlo, qhi in paint:
            work = tree.increment(qlo, qhi-1, 0, N, 0)
            worklog.append(work)
            
        return worklog



# not good
class Solution:
    def amountPainted(self, paint: List[List[int]]) -> List[int]:
        seen = {}
        work_log = []
        
        for start, end in paint:
            work = 0
            
            while start < end:  # 10^5 memory -> 
                if start in seen: start = seen[start]
                else:
                    seen[start] = end
                    work += 1
                    start += 1  # 10^5 time
                    
            work_log.append(work)
                    
        return work_log

In [5]:

class Solution:
    def largestRectangleArea(self, height):
        # height.append(0)
        length = len(height)
        stack = [-1]
        ans = 0
        for i in range(length+1):
            hei = 0 if i == length else height[i]
            while hei < height[stack[-1]]:
                h = height[stack.pop()]
                w = i - stack[-1] - 1
                ans = max(ans, h * w)
            stack.append(i)
        height.pop()
        return ans
height = [2,1,5,6,2,3]
Solution().largestRectangleArea(height)

IndexError: list index out of range

In [2]:
import bisect
def max_profit(start_time, end_time, d_starts, d_ends, d_pays):
    dp_end = [0]
    dp_profit = [0]
    dp_track = [0]
    for end, start, pay in sorted(zip(d_ends, d_starts, d_pays)):
        if end > end_time or start < start_time:
            continue
        idx = bisect.bisect_left(dp_end, start+1)
        profit = dp_profit[idx-1] + pay
        if profit > dp_profit[-1]:
            dp_profit.append(profit)
            dp_end.append(end)
            dp_track.append((start, end, pay, idx-1))
            
    previous = len(dp_track)-1
    path = []
    while previous > 0:
        path.append(dp_track[previous])
        previous = dp_track[previous][-1]
    print(path[::-1])
    print(dp_track)
    print(dp_profit)
    print(dp_end)
    return dp_profit[-1]

def max_profit_k(start_time, end_time, d_starts, d_ends, d_pays, k):
    cur_list = [ (e,s,p, i) for i,(e,s,p) in enumerate(list(zip(d_ends, d_starts, d_pays)))]
    cur_set = set()
    step = 0
    total = 0
    jobs = []
    while cur_list and step < k:
        step += 1
        cur_max, cur_jobs = max_profit_one(start_time, end_time, cur_list)
        cur_set = set([idx for _, _,_,idx in cur_jobs])
        cur_list = [(e,s,p,i) for e,s,p,i in cur_list if i not in cur_set]
        total += cur_max 
        jobs.append(cur_jobs)
    print(f"total: {total}")
    print(f"jobs: {jobs}")
        
    
def max_profit_one(start_time, end_time, deliveries):
    dp_end = [0]
    dp_profit = [0]
    dp_track = [0]
    for end, start, pay, index in sorted(deliveries):
        if end > end_time or start < start_time:
            continue
        idx = bisect.bisect_left(dp_end, start+1)
        profit = dp_profit[idx-1] + pay
        if profit > dp_profit[-1]:
            dp_profit.append(profit)
            dp_end.append(end)
            dp_track.append((start, end, pay,index, idx-1))
            
    previous = len(dp_track)-1
    path = []
    while previous > 0:
        path.append(dp_track[previous][:4])
        previous = dp_track[previous][-1]
    print(path[::-1])
    print(dp_track)
    print(dp_profit)
    print(dp_end)
    return dp_profit[-1], path[::-1]

start_time = 0
end_time = 10
d_starts = [2, 3, 5, 7]
d_ends = [6, 5, 10, 11]
d_pays = [5, 2, 4, 1]

print(max_profit(start_time, end_time, d_starts, d_ends, d_pays))

max_profit_k(start_time, end_time, d_starts, d_ends, d_pays, 2)

[(3, 5, 2, 0), (5, 10, 4, 1)]
[0, (3, 5, 2, 0), (2, 6, 5, 0), (5, 10, 4, 1)]
[0, 2, 5, 6]
[0, 5, 6, 10]
6
[(3, 5, 2, 1), (5, 10, 4, 2)]
[0, (3, 5, 2, 1, 0), (2, 6, 5, 0, 0), (5, 10, 4, 2, 1)]
[0, 2, 5, 6]
[0, 5, 6, 10]
[(2, 6, 5, 0)]
[0, (2, 6, 5, 0, 0)]
[0, 5]
[0, 6]
total: 11
jobs: [[(3, 5, 2, 1), (5, 10, 4, 2)], [(2, 6, 5, 0)]]


In [14]:
class Solution:
    def canReorderDoubled(self, arr):        
        cnt = collections.Counter(arr)
        
        for c in sorted(cnt.keys(), key=abs):
            print(f"c:{c}")
            if cnt[2 * c] < cnt[c]:  # ??? 0 ???
                return False
            cnt[2 * c] -= cnt[c]
        
        return True
    
s = Solution()
print(s.canReorderDoubled([1,2,0,2,2,4]))

c:0
c:1
c:2
False


In [9]:
arr = [3,2,4,1]
print(sorted(arr))
arr.sort()
print(arr)

[1, 2, 3, 4]
[1, 2, 3, 4]


In [8]:
import collections
class Solution:
    def maxTaskAssign(self, tasks, workers, pills, strength):
        tasks = sorted(tasks)
        workers = sorted(workers)
        print(tasks)
        print(workers)

        def check(k):
            top_workers =workers[-k:][::-1]  # biggest workers, decreasing order
            boosts = pills
            worker_deque = collections.deque()
            worker_index = 0
            print(f"k={k}, top_worker={top_workers}, boosts={boosts}")

            for task in tasks[:k][::-1]:  # smallest k tasks, decreasing order
                while worker_index < k and top_workers[worker_index] + strength >= task:
                    print(f"task={task}, append worker {top_workers[worker_index]}")
                    worker_deque.append(top_workers[worker_index])
                    worker_index += 1
                if not worker_deque:
                    return False
                if worker_deque[0] >= task:
                    worker_deque.popleft()
                elif boosts > 0:
                    worker_deque.pop()
                    boosts -= 1
                else:
                    return False
                    

            return True

        beg, end = 0, min(len(workers), len(tasks)) + 1
        while beg + 1 < end:
            mid = (beg + end)//2
            print(f"mid = {mid}")
            if check(mid):
                print(f"check passed {mid}")
                beg = mid
            else:
                print(f"check failed {mid}")
                end = mid

        return beg
    
s = Solution()
print(s.maxTaskAssign([3,2,1],[0,3,3],1, 1))

[1, 2, 3]
[0, 3, 3]
mid = 2
k=2, top_worker=[3, 3], boosts=1
task=2, append worker 3
task=2, append worker 3
check failed 2
mid = 1
k=1, top_worker=[3], boosts=1
task=1, append worker 3
check passed 1
1


In [5]:
import bisect
class Solution:
    def insert(self, intervals, newInterval):
        
        res = []
        
        start = bisect.bisect_left(intervals, [newInterval[0], newInterval[0]])
        print(start)
        # end = bisect.bisect_right(intervals, [newInterval[1], newInterval[1]], key=lambda x: x[0])
        for i in range(start-1, len(intervals)):
            interval = intervals[i]            
            if interval[1] < newInterval[0]:
                res.append(interval)
            elif interval[0] > newInterval[1]:
                res.append(newInterval)
                return intervals[:start-1]+res + intervals[i:]
            else:
                newInterval[0] = min(interval[0], newInterval[0])
                newInterval[1] = max(interval[1], newInterval[1])
                
        res.append(newInterval)
        
        return intervals[:start-1]+res


s = Solution()
print(s.insert([[1,3],[6,9]],[2,5]))

1


TypeError: 'key' is an invalid keyword argument for bisect_right()

In [12]:
def criticalConnections( n, connections):  # add bigger
    graph = [[] for _ in range(n)]
    visit_rank = {}
    result = []
    low_rank = [0]*n
    for a, b in connections:
        graph[a].append(b)
        graph[b].append(a)
    
    def dfs(node, rank, parent):  # update the low 
        print(f" dfs: {node}, {rank}, {parent}")
        visit_rank[node] = rank
        low_rank[node] = rank
        min_rank = rank + 1
        for nb in graph[node]:
            if nb == parent:
                continue
            if nb not in visit_rank:  # nb is first time to visit 
                dfs(nb, rank+1, node)
                low_rank[node] = min(low_rank[node], low_rank[nb])
                if low_rank[nb] > rank:  # if nb can not accessed from other nodes, it is critical connection
                    result.append((min(nb,node), max(nb,node)))
                    print(f"bigger: {(min(nb,node), max(nb,node))}, {low_rank[nb]}, {rank}, {node}, {nb},")
                else:
                    print(f"smaller: {(min(nb,node), max(nb,node))}, {low_rank[nb]}, {rank}, {node}, {nb},")
            elif nb in visit_rank and nb!=parent:  # nb is visited before, which may give node a smaller rank
                low_rank[node] = min(low_rank[node], low_rank[nb])

    
    dfs(0,0, -1)
    return result

print(criticalConnections(4, [[0,1],[1,2],[2,0],[1,3]]))


#  dfs: 0, 0, -1
#  dfs: 1, 1, 0
#  dfs: 2, 2, 1
# smaller: (1, 2), 0, 1, 1, 2,
#  dfs: 3, 2, 1
# bigger: (1, 3), 2, 1, 1, 3,
# smaller: (0, 1), 0, 0, 0, 1,
# [(1, 3)]

 dfs: 0, 0, -1
 dfs: 1, 1, 0
 dfs: 2, 2, 1
smaller: (1, 2), 0, 1, 1, 2,
 dfs: 3, 2, 1
bigger: (1, 3), 2, 1, 1, 3,
smaller: (0, 1), 0, 0, 0, 1,
[(1, 3)]


In [10]:

def criticalConnections( n,connections): # remove smaller
    visited_level = {}  
    # similar to level in level order traversal there is no update for level, 
    # just assign at first time as there is no cycle
    graph = [[] for _ in range(n)]
    edges = set()
    # all the edges initially, and remove all the cycle edges during dfs  
    for n1, n2 in connections:
        graph[n1].append(n2)
        graph[n2].append(n1)
        edges.add((min(n1,n2), max(n1,n2)))
    
    def dfs(node, level, parent):
        print(f" dfs: {node}, {level}, {parent}")
        if node in visited_level: # visited before 
            return visited_level[node]
        visited_level[node] = level  # first and only time set it, at first visit 
        min_level = level + 1 # for next level
        for nb in graph[node]:
            if nb==parent:
                continue                
            recursive_level = dfs(nb, level+1, node) # get the mimimum visited level from all connected nodes
            # nb can be visited before with a smaller level, which is the minimum value we need
            # if no cycle, level+2
            if recursive_level <= level:
                edges.discard((min(nb,node), max(nb,node)))
                print(f"smaller: {(min(nb,node), max(nb,node))}, {recursive_level}, {level}, {node}, {nb},")
            else:
                print(f"bigger: {(min(nb,node), max(nb,node))}, {recursive_level}, {level},  {node}, {nb},")
            min_level = min(min_level, recursive_level)
        return min_level  # is the connected minimum level

    dfs(0, 0, -1)
    return [e for e in edges]


print(criticalConnections(4, [[0,1],[1,2],[2,0],[1,3]]))

#  dfs: 0, 0, -1
#  dfs: 1, 1, 0
#  dfs: 2, 2, 1
#  dfs: 0, 3, 2
# smaller: (0, 2), 0, 2, 2, 0,
# smaller: (1, 2), 0, 1, 1, 2,
#  dfs: 3, 2, 1
# bigger: (1, 3), 3, 1,  1, 3,
# smaller: (0, 1), 0, 0, 0, 1,
#  dfs: 2, 1, 0
# bigger: (0, 2), 2, 0,  0, 2,
# [(1, 3)]

 dfs: 0, 0, -1
 dfs: 1, 1, 0
 dfs: 2, 2, 1
bigger: (1, 2), 3, 1,  1, 2,
 dfs: 3, 2, 1
bigger: (1, 3), 3, 1,  1, 3,
bigger: (0, 1), 2, 0,  0, 1,
[(0, 1), (0, 2), (1, 2), (1, 3)]


In [ ]:
# 6059. Check if There Is a Valid Parentheses String Path
# User Accepted:422
# User Tried:962
# Total Accepted:502
# Total Submissions:1847
# Difficulty:Hard
# A parentheses string is a non-empty string consisting only of '(' and ')'. It is valid if any of the following conditions is true:

# It is ().
# It can be written as AB (A concatenated with B), where A and B are valid parentheses strings.
# It can be written as (A), where A is a valid parentheses string.
# You are given an m x n matrix of parentheses grid. A valid parentheses string path in the grid is a path satisfying all of the following conditions:

# The path starts from the upper left cell (0, 0).
# The path ends at the bottom-right cell (m - 1, n - 1).
# The path only ever moves down or right.
# The resulting parentheses string formed by the path is valid.
# Return true if there exists a valid parentheses string path in the grid. Otherwise, return false.


class Solution:  # time limit exceed
    def hasValidPath(self, grid: List[List[str]]) -> bool:
        if not grid:
            return False
        R = len(grid)
        C = len(grid[0])
        if (R+C-1)%2==1:
            return False
        total_left = (R+C-1)//2
        def dfs(row, col, left):
            if row < 0 or row > R-1 or col < 0 or col > C-1:
                return False
            if row == R-1 and col == C-1:
                if (grid[row][col] == ')' and left == 1):
                    return True
                else:
                    return False
            if grid[row][col] == '(':
                left += 1
                if left > total_left:
                    return False
            else:
                left -=1
                if left < 0:
                    return False
            if dfs(row+1, col, left) or dfs(row, col+1, left):
                return True
        return dfs(0,0,0)

In [ ]:
# 6057. Count Nodes Equal to Average of Subtree
# User Accepted:3569
# User Tried:3915
# Total Accepted:3839
# Total Submissions:4617
# Difficulty:Medium
# Given the root of a binary tree, return the number of nodes where the value of the node is equal to the average of the values in its subtree.

# Note:

# The average of n elements is the sum of the n elements divided by n and rounded down to the nearest integer.
# A subtree of root is a tree consisting of root and all of its descendants.


# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def averageOfSubtree(self, root: Optional[TreeNode]) -> int:
        def help(root):
            if not root:
                return 0, 0, 0
            left_total, left_nodes, left_result = help(root.left)
            right_total, right_nodes, right_result = help(root.right)
            total = root.val + left_total + right_total
            nodes = left_nodes + right_nodes + 1
            average = total // nodes
            result = left_result+right_result
            result += 1 if average == root.val else 0
            return total, nodes, result
        return help(root)[2]

In [34]:
    """6058. Count Number of Texts
User Accepted:983
User Tried:1285
Total Accepted:1042
Total Submissions:2137
Difficulty:Medium
Alice is texting Bob using her phone. The mapping of digits to letters is shown in the figure below.


In order to add a letter, Alice has to press the key of the corresponding digit i times, where i is the position of the letter in the key.

For example, to add the letter 's', Alice has to press '7' four times. Similarly, to add the letter 'k', Alice has to press '5' twice.
Note that the digits '0' and '1' do not map to any letters, so Alice does not use them.
However, due to an error in transmission, Bob did not receive Alice's text message but received a string of pressed keys instead.

For example, when Alice sent the message "bob", Bob received the string "2266622".
Given a string pressedKeys representing the string received by Bob, return the total number of possible text messages Alice could have sent.

Since the answer may be very large, return it modulo 109 + 7.

    Returns:
        _type_: _description_
    """

class Solution:
    def countTexts(self, pressedKeys: str) -> int:
        # 2 - 6 8 -> 123
        # 7 9 -> 1234
        three = "234568"
        four = "79"
        L = len(pressedKeys)
        def dfs(idx):
            if idx in memo:
                return memo[idx]
            res = 0
            ran = 3 if pressedKeys[idx] in three else (4 if pressedKeys[idx] in four else 0)
            for i in range(idx, min(L, idx+ran)):
                if pressedKeys[i] == pressedKeys[idx]:
                    res += dfs(i+1) % (10**9+7)
                    res %= (10**9+7)
                else:
                    break
            memo[idx] = res
            return res
            
        memo = {len(pressedKeys):1}
        res = dfs(0)
        print(memo)
        return res
    
print(Solution().countTexts("344644885"))
# Output:
# 10
# Expected:
# 8

# 344644885
# 012345678
# {9: 1, 
#  8: 1, 
#  7: 1, 
#  6: 2, 
#  5: 2, 
#  4: 4, 
#  3: 4, 
#  2: 6, 
#  1: 10, 
#  0: 10}
# 10
                

{9: 1, 8: 1, 7: 1, 6: 2, 5: 2, 4: 4, 3: 4, 2: 6, 1: 10, 0: 10}
10


In [31]:
    """6056. Largest 3-Same-Digit Number in String
User Accepted:5853
User Tried:6395
Total Accepted:6588
Total Submissions:11491
Difficulty:Easy
You are given a string num representing a large integer. An integer is good if it meets the following conditions:

It is a substring of num with length 3.
It consists of only one unique digit.
Return the maximum good integer as a string or an empty string "" if no such integer exists.

Note:

A substring is a contiguous sequence of characters within a string.
There may be leading zeroes in num or a good integer.

    Returns:
        _type_: _description_
    """


class Solution:
    def largestGoodInteger(self, num: str) -> str:
        if not num:
            return ""
        L = len(num)
        if L < 3:
            return ""
        res = -1
        for i in range(2, L):
            if num[i] == num[i-1] == num[i-2] and num[i] in '0123456789':
                res = max(res, ord(num[i]) - ord('0'))
        return str(res)*3 if res!=-1 else ""
    
print(Solution().largestGoodInteger('12333456777'))

777


In [25]:
import collections
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        # top down approach: prev is out side of [i,j], <= i and/or >= j
        def dp(i, j, prev):
            ## dfs(i, j, prev) is the subproblem from i to j with prev char
            if (i, j, prev) in memo:
                return memo[i, j, prev]
            
            # return 0 for singlet string 
            if i >= j:  # i==j is odd length of palindromic string
                return 0
            
            # specify the condition
            if s[i] == s[j] and s[i] != prev:
                memo[i, j, prev] = 2 + dp(i + 1, j - 1, s[i])
                print(f"TD  Total: {memo[i, j, prev]:03d}\tExtend by two: {i:03d}\t{j:03d}\t{s[i]}")
            else:
                memo[i, j, prev] = max(dp(i + 1, j, prev), dp(i, j - 1, prev))
                
            return memo[i, j, prev]
        memo = {}
        res = dp(0, len(s) - 1, '')
        # print(memo)
        return res
    
class Solution2:
    def longestPalindromeSubseq(self, s: str) -> int:
        if not s:
            return 0
        L = len(s)
        dp = collections.defaultdict(lambda: collections.defaultdict(int))
        for j in range(L): # j++
            for i in range(j, -1, -1): # i--, i<j
                if i==j:
                    dp[i,j][''] = 0
                elif s[i] == s[j]:
                    if j-i==1:
                        dp[i,j][s[i]] = 2
                        # print(f"BU  Total: {dp[i,j][s[i]]:03d}\tExtend by two: {i:03d}\t{j:03d}\t{s[i]}")
                    else: # j-i>=2
                        for pre in dp[i+1, j-1]:
                            if pre == s[i]:
                                dp[i,j][s[i]] = max(dp[i,j][s[i]],dp[i+1,j-1][pre])
                            else:
                                dp[i,j][s[i]] = max(dp[i,j][s[i]],dp[i+1,j-1][pre]+2)
                            # print(f"BU  Total: {dp[i,j][s[i]]:03d}\tExtend by two: {i:03d}\t{j:03d}\t{s[i]}")
                else:
                    for pre in set(dp[i+1,j].keys()) | set(dp[i,j-1].keys()):
                        
                        dp[i,j][pre] = max(dp[i,j][pre],dp[i+1,j][pre], dp[i,j-1][pre])
                        # print(i,j,pre,dp[i,j][pre])
        # print(dp)
        return max(dp[0,L-1].values())

solution = Solution()
s="rggggr"
s1="wvtjiwbftzchwsnxkqtuykvunmsddpnnbortmenayrgglcmvieqvybuibkikhwlveafjscdyzhopflyhzxtbwqiahgptmhbiimksazuulkcahmgzauuyggrhpnfwmwgblcwpanceziqrbxokjmtijpaluqcgelg"
solution.longestPalindromeSubseq(s)

solution2 = Solution2()
solution2.longestPalindromeSubseq(s)


TD  Total: 002	Extend by two: 001	004	g
TD  Total: 004	Extend by two: 000	005	r
0 1  0
BU  Total: 002	Extend by two: 001	002	g
0 2  0
0 2 g 2
BU  Total: 002	Extend by two: 002	003	g
BU  Total: 002	Extend by two: 001	003	g
0 3  0
0 3 g 2
BU  Total: 002	Extend by two: 003	004	g
BU  Total: 002	Extend by two: 002	004	g
BU  Total: 002	Extend by two: 001	004	g
0 4  0
0 4 g 2
4 5  0
3 5  0
3 5 g 2
2 5  0
2 5 g 2
1 5  0
1 5 g 2
BU  Total: 004	Extend by two: 000	005	r
BU  Total: 004	Extend by two: 000	005	r
defaultdict(<function Solution2.longestPalindromeSubseq.<locals>.<lambda> at 0x104a97310>, {(0, 0): defaultdict(<class 'int'>, {'': 0}), (1, 1): defaultdict(<class 'int'>, {'': 0}), (0, 1): defaultdict(<class 'int'>, {'': 0, 'g': 0}), (2, 2): defaultdict(<class 'int'>, {'': 0}), (1, 2): defaultdict(<class 'int'>, {'g': 2, '': 0}), (0, 2): defaultdict(<class 'int'>, {'': 0, 'g': 2}), (3, 3): defaultdict(<class 'int'>, {'': 0}), (2, 3): defaultdict(<class 'int'>, {'g': 2}), (1, 3): defaultdict

4

In [ ]:
def unboundedBackpack(wt, V, m):
    
    f = [0] * (m + 1)
    for j in range(m + 1): # quota
        for i in range(len(wt)):  # items (unlimited)  
            if j >= wt[i]:
                f[j] = max(f[j], f[j - wt[i]] + V[i])
            # print(f)
    print(f)         
    return f[-1]

def unboundedBackpack2(wt, V, m):
    
    f = [0] * (m + 1)
    for i in range(len(wt)):  # items (unlimited) 
        for j in range(m + 1): # quota
            if j >= wt[i]:
                f[j] = max(f[j], f[j - wt[i]] + V[i])
            # print(f)
    print(f)         
    return f[-1]
W = 10
val = [1, 3, 2] 
wt = [1, 8, 3] 
print(unboundedBackpack(wt,val,W))
print(unboundedBackpack2(wt,val,W))

In [8]:
a = list((12,2))
a.sort()
print(a)

[2, 12]


In [65]:
def splitString(s):
        # write your code here
        stack = []
        stack.append((0,list()))
        res = []
        if not s or not len(s):
            return res
        while stack:
            idx, path = stack.pop()
            if idx == len(s):
                res.append(path[:])
            elif idx < len(s):
                path1 = list(path)
                path1.append(s[idx:idx+1])
                stack.append((idx+1, path1))
                if idx < len(s) - 1:
                    path2 = list(path)
                    path2.append(s[idx:idx+2])
                    stack.append((idx+2, path2))
        return res
print(splitString("123"))

[['12', '3'], ['1', '23'], ['1', '2', '3']]


In [22]:
for i in 0,1:
    for one in 0,1:
        for two in 0,1:
            preone = one
            pretwo = two
            one = (one ^ i) & (~two)
            two = (two ^ i) & (~one)
            print(f"one={preone}, two={pretwo}, i={i}, one={one}, two={two}")


one=0, two=0, i=0, one=0, two=0
one=0, two=1, i=0, one=0, two=1
one=1, two=0, i=0, one=1, two=0
one=1, two=1, i=0, one=0, two=1
one=0, two=0, i=1, one=1, two=0
one=1, two=1, i=1, one=0, two=0
one=1, two=0, i=1, one=0, two=1
one=0, two=1, i=1, one=0, two=0


In [20]:
print(f"{(1<<0):0b}")

1


In [15]:
def hasAlternatingBits(n: int) -> bool:
        print(f"{n:0b}")
        right_first_bit = (1 & n)
        print(f"{right_first_bit:0b}")
        flip = right_first_bit ^ 1 # flip only first bit from right
        print(f"{flip:0b}")
        left_shift = n << 1 # left shift, the last bit is 0 (cleared)
        print(f"{left_shift:0b}")
        set_last_bit = left_shift | flip # flip of the original last bit
        print(f"{set_last_bit:0b}")
        print(f"{set_last_bit ^ n:0b}")
        print(f"{(set_last_bit ^ n) +1:0b}")
        power2 = (set_last_bit ^ n) + 1
        return power2 & (power2 -1) == 0 # all bits are different 

hasAlternatingBits(-5)

-101
1
0
-1010
-1010
1101
1110


False

In [7]:
for ch in 'ACTG':
    print((ord(ch) - ord('A') +1)%5)

1
3
0
2


In [17]:
for i in range(2,3):
    print(i)

2


In [15]:
def gcd(a, b):
        
        if b == 0:
            return a
        return gcd(b, a % b)
print(gcd(-15, -10), gcd(15,10))
print(gcd(-15, 10), gcd(15,-10))
print(gcd(10, -15), gcd(-10,15))

-5 5
5 -5
-5 5


In [12]:
print(15 // 5, -15 // 5)

3 -3


In [9]:
import collections
visited = collections.defaultdict(lambda: collections.defaultdict(int))

In [ ]:
from typing import (
    List,
)
import bisect, collections
class Solution:
    """
    @param intervals: the intervals
    @param rooms: the sum of rooms
    @param ask: the ask
    @return: true or false of each meeting
    """
    def meeting_room_i_i_i(self, intervals: List[List[int]], rooms: int, ask: List[List[int]]) -> List[bool]:
        count = collections.defaultdict(int)
        sum = collection.defaultdict(int)
        startMin = float('inf')
        endMax = float('-inf')
        for start, end in intervals:
            count[start] += 1
            count[end] -= 1
            startMin = min(startMin, start)
            endMax = max(endMax, end)
        for start, end in ask:
            startMin = min(startMin, start)
            endMax = max(endMax, end)
        cnt = 0
        for i in range(startMin, endMax + 1):
            cnt += count[i]
            if (cnt < rooms):
                sum[i] = 0
            else:
                sum[i] = 1
        
        for i in range(startMin, endMax + 1):
            sum[i] += sum[i-1]
            
        result = []
        for start, end in ask:
            if sum[start-1] == sum[end-1]:
                result.append(True)
            else:
                result.append(False)
        return result
                
        
            
            